In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests

from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, Legend, LogColorMapper, GeoJSONDataSource, LinearColorMapper, ColorBar, Range1d, NumeralTickFormatter, HoverTool, LabelSet, Panel, Tabs, Slider, CustomJS
from bokeh.models.widgets import TableColumn, DataTable, NumberFormatter, Dropdown, Select, RadioButtonGroup
from bokeh.palettes import Category20c, Spectral, GnBu, brewer, PRGn, RdYlGn
from bokeh.io import curdoc, output_notebook, show, output_file
from bokeh.layouts import row, column, gridplot
from bokeh.palettes import Viridis6 as palette
from bokeh.transform import cumsum
from bokeh.resources import INLINE
import bokeh.io

import geopandas as gpd
import pycountry
import pycountry_convert as pc
import json

from powerbiclient import QuickVisualize, get_dataset_config, Report
from powerbiclient.authentication import DeviceCodeLoginAuthentication

from rich.console import Console

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv(r'data/RESHARE_20231208-072420.csv', encoding_errors='replace', header=2)

In [4]:
ren_df = pd.read_csv(r'data/REGEN_20231208-161335.csv', encoding_errors='replace', header=1)

In [5]:
df

,Region/country/area,Indicator,2018,2019,2020,2021,2022
0,World,RE share of electricity capacity (%),33.16,34.60,36.66,38.42,40.26
1,World,RE share of electricity generation (%),24.87,25.85,27.65,27.79,0.00
2,Africa,RE share of electricity capacity (%),21.61,21.71,22.51,22.94,23.61
3,Africa,RE share of electricity generation (%),19.86,20.66,22.31,22.74,0.00
4,Algeria,RE share of electricity capacity (%),3.09,2.80,2.63,2.28,2.26
...,...,...,...,...,...,...,...
463,Suriname,RE share of electricity generation (%),56.19,51.35,39.49,53.68,0.00
464,Uruguay,RE share of electricity capacity (%),75.80,76.00,76.02,76.06,76.12
465,Uruguay,RE share of electricity generation (%),97.32,98.04,93.92,84.53,0.00
466,Venezuela (Bolivarian Republic of),RE share of electricity capacity (%),48.82,49.14,48.68,48.68,48.68


In [6]:
gen_df = df.query('Indicator=="RE share of electricity generation (%)"')

In [7]:
gen_df

,Region/country/area,Indicator,2018,2019,2020,2021,2022
1,World,RE share of electricity generation (%),24.87,25.85,27.65,27.79,0.0
3,Africa,RE share of electricity generation (%),19.86,20.66,22.31,22.74,0.0
5,Algeria,RE share of electricity generation (%),1.02,1.03,1.06,0.91,0.0
7,Angola,RE share of electricity generation (%),69.76,71.87,73.70,73.70,0.0
9,Benin,RE share of electricity generation (%),1.64,0.40,0.40,0.22,0.0
...,...,...,...,...,...,...,...
459,Peru,RE share of electricity generation (%),60.89,60.51,64.38,61.67,0.0
461,South Georgia and S Sandwich Islands,RE share of electricity generation (%),98.60,98.96,98.96,100.00,0.0
463,Suriname,RE share of electricity generation (%),56.19,51.35,39.49,53.68,0.0
465,Uruguay,RE share of electricity generation (%),97.32,98.04,93.92,84.53,0.0


In [8]:
ren_df['unit'] = 'gwh'

ren_df.columns = [
    'country',
    'technology',
    '2018',
    '2019',
    '2020',
    '2021',
    'unit'
]
ren_df['country'] = ren_df['country'].copy().apply(lambda x: x.lower())
ren_df['technology'] = ren_df['technology'].copy().apply(lambda x: x.lower())

ren_df

,country,technology,2018,2019,2020,2021,unit
0,world,total renewable energy,6627718.734,6994720.276,7455905.333,7857802.526,gwh
1,world,hydropower,4306776.270,4352825.868,4477721.630,4400679.013,gwh
2,world,renewable hydropower,4189346.361,4238697.906,4357351.497,4275374.687,gwh
3,world,pumped storage,117429.909,114127.962,120370.133,125304.326,gwh
4,world,marine,1021.241,995.175,983.975,970.024,gwh
...,...,...,...,...,...,...,...
4726,venezuela (bolivarian republic of),bagasse,0.000,0.000,0.000,0.000,gwh
4727,venezuela (bolivarian republic of),other solid biofuels,0.000,0.000,0.000,0.000,gwh
4728,venezuela (bolivarian republic of),liquid biofuels,0.000,0.000,0.000,0.000,gwh
4729,venezuela (bolivarian republic of),biogas,0.000,0.000,0.000,0.000,gwh


In [9]:
gen_df.drop(['Indicator', '2022'], axis=1, inplace=True)
gen_df['technology'] = 'total_perc'
gen_df['unit'] = 'perc'

gen_df.columns = [
    'country',
    '2018',
    '2019',
    '2020',
    '2021',
    'technology',
    'unit'
]

gen_df['country'] = gen_df['country'].copy().apply(lambda x: x.lower())
gen_df['technology'] = gen_df['technology'].copy().apply(lambda x: x.lower())

gen_df

,country,2018,2019,2020,2021,technology,unit
1,world,24.87,25.85,27.65,27.79,total_perc,perc
3,africa,19.86,20.66,22.31,22.74,total_perc,perc
5,algeria,1.02,1.03,1.06,0.91,total_perc,perc
7,angola,69.76,71.87,73.70,73.70,total_perc,perc
9,benin,1.64,0.40,0.40,0.22,total_perc,perc
...,...,...,...,...,...,...,...
459,peru,60.89,60.51,64.38,61.67,total_perc,perc
461,south georgia and s sandwich islands,98.60,98.96,98.96,100.00,total_perc,perc
463,suriname,56.19,51.35,39.49,53.68,total_perc,perc
465,uruguay,97.32,98.04,93.92,84.53,total_perc,perc


In [10]:
country_out = {
    't�rkiye': 'turkiye',
    'r�union': 'reunion',
    'iran (islamic republic of)': 'iran',
    "cote d'ivoire": 'ivory coast',
    'bonaire, sint eustatius and saba': 'caribbean netherlands',
    'viet nam': 'vietnam',
    'south georgia and s sandwich islands': 'south georgia island',
    'venezuela (bolivarian republic of)': 'venezuela'
}

In [11]:
gen_df.country.unique().tolist()

['world',
 'africa',
 'algeria',
 'angola',
 'benin',
 'botswana',
 'burkina faso',
 'burundi',
 'cabo verde',
 'cameroon',
 'central african republic',
 'chad',
 'comoros',
 'democratic republic of the congo',
 'congo',
 "cote d'ivoire",
 'djibouti',
 'egypt',
 'equatorial guinea',
 'eritrea',
 'eswatini',
 'ethiopia',
 'gabon',
 'gambia',
 'ghana',
 'guinea',
 'guinea-bissau',
 'kenya',
 'lesotho',
 'liberia',
 'libya',
 'madagascar',
 'malawi',
 'mali',
 'mauritania',
 'mauritius',
 'mayotte',
 'morocco',
 'mozambique',
 'namibia',
 'niger',
 'nigeria',
 'reunion',
 'rwanda',
 'sao tome and principe',
 'senegal',
 'seychelles',
 'sierra leone',
 'somalia',
 'south africa',
 'south sudan',
 'sudan',
 'united republic of tanzania',
 'togo',
 'tunisia',
 'uganda',
 'zambia',
 'zimbabwe',
 'asia',
 'afghanistan',
 'bangladesh',
 'bhutan',
 'brunei darussalam',
 'cambodia',
 'china',
 'china, hong kong sar',
 'chinese taipei',
 'india',
 'indonesia',
 'japan',
 'kazakhstan',
 "democratic

In [12]:
en_df = pd.concat([ren_df, gen_df])

In [13]:
np.max([3,52,3]+[3,2,6])

52

### Percentage sustainable energy generated in the world

In [14]:
round(en_df.query('country=="world" and technology==["total_perc", "total renewable energy"]'),2)

,country,technology,2018,2019,2020,2021,unit
0,world,total renewable energy,6627718.73,6994720.28,7455905.33,7857802.53,gwh
1,world,total_perc,24.87,25.85,27.65,27.79,perc


### Percentage sustainable energy generated in the world

In [32]:
def country_data(data, name):
    """Percentage energy data from each country"""
    data = data.query('country==@name and technology!="total_perc"')
    
    dict_co = en_df.query('country==@name and technology!="total_perc" and technology=="total renewable energy"')
    
    data['2018'] = data['2018'].apply(lambda s: round(s*100/dict_co['2018'], 2)).copy()
    data['2019'] = data['2019'].apply(lambda s: round(s*100/dict_co['2019'], 2)).copy()
    data['2020'] = data['2020'].apply(lambda s: round(s*100/dict_co['2020'], 2)).copy()
    data['2021'] = data['2021'].apply(lambda s: round(s*100/dict_co['2021'], 2)).copy()
    
    return data[['technology', '2018', '2019', '2020', '2021']]

### World

In [33]:
country_data(en_df,"world")

,technology,2018,2019,2020,2021
0,total renewable energy,100.00,100.00,100.00,100.00
1,hydropower,64.98,62.23,60.06,56.00
2,renewable hydropower,63.21,60.60,58.44,54.41
3,pumped storage,1.77,1.63,1.61,1.59
4,marine,0.02,0.01,0.01,0.01
5,wind,18.99,20.19,21.30,23.39
6,onshore wind energy,17.96,18.98,19.91,21.64
7,offshore wind energy,1.03,1.21,1.40,1.75
8,solar,8.49,9.92,11.30,13.16
9,solar photovoltaic,8.31,9.71,11.12,12.98


### European Union

In [35]:
country_data(en_df, "european union")

,technology,2018,2019,2020,2021
3458,total renewable energy,100.00,100.00,100.00,100.00
3459,hydropower,39.41,35.34,35.43,34.86
3460,renewable hydropower,36.61,32.75,32.77,32.40
3461,pumped storage,2.80,2.59,2.66,2.46
3462,marine,0.05,0.05,0.05,0.05
3463,wind,34.02,37.46,37.46,35.90
3464,onshore wind energy,30.63,33.35,32.99,31.47
3465,offshore wind energy,3.40,4.11,4.47,4.43
3466,solar,12.05,12.69,13.71,15.25
3467,solar photovoltaic,11.53,12.11,13.24,14.77


### Asia

In [36]:
country_data(en_df, "asia")

,technology,2018,2019,2020,2021
1121,total renewable energy,100.00,100.00,100.00,100.00
1122,hydropower,67.03,64.21,62.14,56.10
1123,renewable hydropower,65.00,62.39,60.37,54.28
1124,pumped storage,2.03,1.82,1.77,1.82
1125,marine,0.02,0.02,0.01,0.01
1126,wind,16.62,17.08,17.92,21.87
1127,onshore wind energy,16.26,16.66,17.41,20.28
1128,offshore wind energy,0.36,0.43,0.51,1.58
1129,solar,11.04,12.86,14.27,16.08
1130,solar photovoltaic,11.02,12.84,14.26,16.07


### Africa

In [37]:
country_data(en_df, "africa")

,technology,2018,2019,2020,2021
19,total renewable energy,100.00,100.00,100.00,100.00
20,hydropower,86.63,84.97,83.11,81.91
21,renewable hydropower,83.63,81.90,80.23,79.26
22,pumped storage,3.00,3.07,2.88,2.65
23,marine,0.00,0.00,0.00,0.00
24,wind,4.84,5.58,5.75,6.35
25,onshore wind energy,4.84,5.58,5.75,6.35
26,offshore wind energy,0.00,0.00,0.00,0.00
27,solar,6.30,7.47,9.39,10.09
28,solar photovoltaic,4.79,5.40,7.63,8.33


### Asia

In [38]:
country_data(en_df, "north america")

,technology,2018,2019,2020,2021
3762,total renewable energy,100.00,100.00,100.00,100.00
3763,hydropower,59.63,57.10,54.14,50.07
3764,renewable hydropower,57.88,55.44,52.55,48.57
3765,pumped storage,1.75,1.66,1.59,1.50
3766,marine,0.00,0.00,0.00,0.00
3767,wind,26.12,27.72,29.79,31.73
3768,onshore wind energy,26.12,27.71,29.78,31.72
3769,offshore wind energy,0.01,0.01,0.01,0.01
3770,solar,7.50,8.92,10.50,12.85
3771,solar photovoltaic,7.18,8.63,10.24,12.62


### Ethiopia

In [34]:
country_data(en_df, "ethiopia")

,technology,2018,2019,2020,2021
399,total renewable energy,100.00,100.00,100.00,100.00
400,hydropower,95.64,93.83,95.57,94.47
401,renewable hydropower,95.64,93.83,95.57,94.47
402,pumped storage,0.00,0.00,0.00,0.00
403,marine,0.00,0.00,0.00,0.00
404,wind,4.02,5.83,4.04,5.17
405,onshore wind energy,4.02,5.83,4.04,5.17
406,offshore wind energy,0.00,0.00,0.00,0.00
407,solar,0.15,0.14,0.21,0.21
408,solar photovoltaic,0.15,0.14,0.21,0.21


### Brazil

In [39]:
country_data(en_df, "brazil")

,technology,2018,2019,2020,2021
4484,total renewable energy,100.00,100.00,100.00,100.00
4485,hydropower,78.53,77.19,75.79,71.47
4486,renewable hydropower,78.53,77.19,75.79,71.47
4487,pumped storage,0.00,0.00,0.00,0.00
4488,marine,0.00,0.00,0.00,0.00
4489,wind,9.79,10.86,10.91,14.24
4490,onshore wind energy,9.79,10.86,10.91,14.24
4491,offshore wind energy,0.00,0.00,0.00,0.00
4492,solar,0.70,1.29,2.06,3.30
4493,solar photovoltaic,0.70,1.29,2.06,3.30
